# 30 Days of Pandas Challenge

This is a collection of solutions to the [30 Days of Pandas](https://leetcode.com/studyplan/30-days-of-pandas/) challenge on Leetcode. The solutions are organized by how they appear in the challenge. Multiple solutions to the same problem may exist, as I sometimes submit different solutions for the same problem. The solutions are written in Python and use the Pandas library.

In [2]:
import pandas as pd

## Data Filtering

### 595 Big Countries

In [3]:
def big_countries(w: pd.DataFrame) -> pd.DataFrame:
    return w[(w['area'] >= 3_000_000) | (w['population'] >= 25_000_000)][['name', 'population', 'area']]

### 1757 Recyclable and Low Fat Products

In [4]:
def find_products(p: pd.DataFrame) -> pd.DataFrame:
    return p[(p['low_fats'] == 'Y') & (p['recyclable'] == 'Y')][['product_id']]

### 183 Customers Who Never Order

In [5]:
# - Using `isin` to filter customers who have placed orders with `~` to negate the condition
# - Use `rename` to change column names, and `inplace` to modify the DataFrame directly without reassigning it

def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    customers = customers[~customers['id'].isin(orders['customerId'])][['name']]
    customers.rename(columns={'name': 'Customers'}, inplace=True)
    return customers

### 1148 Article Views I

In [6]:
# - Filtering, renaming and dropping duplicates with sorting

def article_views(views: pd.DataFrame) -> pd.DataFrame:
    views = views[views['viewer_id'] == views['author_id']][['author_id']]
    views = views.rename(columns={'author_id': 'id'})
    return views.drop_duplicates().sort_values(by='id')

## String Methods

### 1683 Invalid Tweets

In [7]:
def invalid_tweets(tweets: pd.DataFrame) -> pd.DataFrame:
    return tweets[tweets['content'].str.len() > 15][['tweet_id']]

### 1873 Calculate Special Bonus (! a)

In [8]:
# Test case
employees = pd.DataFrame({
    'employee_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'salary': [5000, 6000, 7000, 8000, 9000]
})

In [9]:
# Approach 1: Using `apply` with a lambda function to calculate the bonus

def calculate_special_bonus(employees: pd.DataFrame) -> pd.DataFrame:
    employees['bonus'] = employees.apply(
        lambda row: row['salary'] if (row['employee_id'] % 2 == 1 and not row['name'].startswith('M')) else 0,
        axis=1
    )
    return employees[['employee_id', 'bonus']].sort_values('employee_id')

result_1 = calculate_special_bonus(employees)
result_1

,employee_id,bonus
0,1,5000
1,2,0
2,3,7000
3,4,0
4,5,9000


In [10]:
# Approach 2: Using `loc` for index locating with boolean indexing

def calculate_special_bonus(employees: pd.DataFrame) -> pd.DataFrame:
    employees['bonus'] = 0 # Intialize the 'bonus' column with 0 (otherwise we would get NaN values)
    employees.loc[ # Locate the rows with conditions satisfies for the 'bonus' column
        (employees['employee_id'] % 2 == 1) & 
        (~employees['name'].str.startswith('M')), 
        'bonus'
    ] = employees['salary'] # Assign the salary to the 'bonus' column where conditions are met
    return employees[['employee_id', 'bonus']].sort_values('employee_id')

result_2 = calculate_special_bonus(employees)
result_2

,employee_id,bonus
0,1,5000
1,2,0
2,3,7000
3,4,0
4,5,9000


### 1667 Fix Names in a Table (! a)

In [11]:
# Approach 1: Own attempt with row-wise `apply`

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    users['name'] = users.apply(
        lambda row: row['name'][0].upper() + row['name'][1:].lower(), axis=1
    )
    return users.sort_values(by='user_id')

In [12]:
# Approach 2: Using `capitalize` method for strings and not bothering with `apply`
# - In this case, we can simply re-assign the `name` column with the capitalized version of each name
# - `apply` would be more suited if transformations involved e.g. filtering or complex logic

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    users['name'] = users['name'].str.capitalize()
    return users.sort_values(by='user_id')

### 1517 Find Users With Valid E-Mails (u)

In [13]:
def valid_emails(users: pd.DataFrame) -> pd.DataFrame:
    mask = r'^[a-zA-Z][a-zA-Z0-9_\-\.]*@leetcode\.com$'
    users = users[users['mail'].str.match(mask)]
    return users

### 1527 Patients With a Condition

In [14]:
# Test case
patients = pd.DataFrame({
    'patient_id': [1, 2, 3, 4, 5],
    'conditions': ['NONE DIAB1', 'DIAB2', 'DIAB1', 'HYPER', 'DIAB1']
})

In [15]:
# Using `contains` instead of `match` for searching over a string  (match is only for exact matches at the start of the string)
# In regex, `^` is equivalent to Python's `startswith`, and `$` is equivalent to `endswith`

def find_patients(patients: pd.DataFrame) -> pd.DataFrame:
    mask = r'(^|\s)DIAB1'
    patients = patients[patients['conditions'].str.contains(mask)]
    return patients

patients_filtered = find_patients(patients)
patients_filtered

C:\Users\murisph\AppData\Local\Temp\ipykernel_2124\3644232046.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  patients = patients[patients['conditions'].str.contains(mask)]


,patient_id,conditions
0,1,NONE DIAB1
2,3,DIAB1
4,5,DIAB1


## Data Manipulation

### 177 Nth Highest Salary (!)

In [16]:
# Test case
'''
| id | salary |
| -- | ------ |
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
'''
employee = pd.DataFrame({
    'id': [1, 2, 3],
    'salary': [100, 200, 300]
})
argument = 1

In [17]:
# Approach 1: My solution

def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:
    result = pd.DataFrame(columns=[f'getNthHighestSalary({N})'])
    employee.drop_duplicates(subset='salary', inplace=True)
    if len(employee) < N or N <= 0: 
        result.loc[0] = None
        return result
    employee.sort_values(by='salary', ascending=False, inplace=True)
    result.loc[0] = employee.iloc[N-1]['salary']
    return result

In [18]:
# Approach 2: Faster solution with initial filtering and conditions for 'salary', and then create DataFrame

def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:
    unique_salaries = employee['salary'].drop_duplicates().sort_values(ascending=False).reset_index(drop=True)

    if len(unique_salaries) >= N and N > 0:
        nth_salary = unique_salaries.iloc[N-1]
    else:
        nth_salary = None

    return pd.DataFrame({f'getNthHighestSalary({N})': [nth_salary]})


result = nth_highest_salary(employee, argument)

### 176 Second Highest Salary (!)

In [42]:
data = [[1, 100], [2, 100], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})

In [ ]:
# Approach 1: Using sort - O(n log n)

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    employee = employee.drop_duplicates(['salary'])
    if len(employee) < 2:
        second_highest = None
    else:
        second_highest = employee['salary'].sort_values(ascending=False).iloc[1]
    return pd.DataFrame({'SecondHighestSalary': [second_highest]})

second_highest = second_highest_salary(employee)
second_highest

   id  salary
0   1     100
2   3     300


,SecondHighestSalary
0,100


In [ ]:
# Approach 2: Using two max() and filtering - O(n)

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    highest = employee['salary'].max()
    second_highest = employee[employee['salary'] < highest]['salary'].max()
    return pd.DataFrame({'SecondHighestSalary': [second_highest]})

second_highest = second_highest_salary(employee)
second_highest


,SecondHighestSalary
0,100


### 184 Department Highest Salary

In [105]:
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [ ]:
# Approach 1: Using merge and groupby
# - merge() works to combine tables. Requires specifying how to JOIN, and names of columns to join in each table
# - groupby() can group e.g. by unique row entries in a column. The result is a groupby object with slightly different rules for data manipulation compared to regular DataFrame objects.

def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    merged = employee.merge(department, how='left', left_on='departmentId', right_on='id')
    grouped = merged.groupby(by='departmentId')
    merged_highest = merged[grouped['salary'].transform('max') == merged['salary']]
    result = merged_highest[['name_y', 'name_x', 'salary']].rename(
        columns={
            'name_y': 'Department',
            'name_x': 'Employee',
            'salary': 'Salary'
        }
    )
    return result
    
a = department_highest_salary(employee, department)
a

   id_x name_x  salary  departmentId  id_y name_y
0     1    Joe   70000             1     1     IT
1     2    Jim   90000             1     1     IT
2     3  Henry   80000             2     2  Sales
3     4    Sam   60000             2     2  Sales
4     5    Max   90000             1     1     IT


,Department,Employee,Salary
1,IT,Jim,90000
2,Sales,Henry,80000
4,IT,Max,90000


In [ ]:
# 